<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/Smartphone_screeentime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# smart phone screentime

- timestamp
- survey
- screenshot ??

# Import Data

## Import Json data from firebase

In [ ]:
import firebase_admin
from firebase_admin import credentials, db
import csv
import pandas as pd

In [ ]:
# Initialize Firebase
cred = credentials.Certificate('path/to/your/serviceAccountKey.json')
firebase_admin.initialize_app(cred, {
    'databaseURL': 'your-database-url'
})

def firebase_to_csv(ref_path, output_file):
    # Get reference to database path
    ref = db.reference(ref_path)

    # Get data
    data = ref.get()

    # Convert to dataframe
    df = pd.DataFrame.from_dict(data, orient='index')

    # Save to CSV
    df.to_csv(output_file)

# Example usage
firebase_to_csv('/your/data/path', 'output.csv')

### working on JSON data

In [ ]:
|import json
import pandas as pd
from datetime import datetime

In [ ]:
pilot_json = open('pilot_data_1.json', encoding = 'utf-8')
data_dict = json.load(pilot_json)
data_dict

{'te001': [None,
  None,
  {'newValue': 1, 'setting': 'isColorEnabled', 'timestamp': 1730148797953},
  {'newValue': 1, 'setting': 'isYellowEnabled', 'timestamp': 1730148807839},
  {'newValue': 0, 'setting': 'isYellowEnabled', 'timestamp': 1730148809787},
  {'newValue': 0, 'setting': 'isColorEnabled', 'timestamp': 1730148810194},
  {'newValue': 3, 'setting': 'thresholdInMinute', 'timestamp': 1730148813022},
  {'newValue': 0, 'setting': 'thresholdInMinute', 'timestamp': 1730148813671},
  {'newValue': 10, 'setting': 'thresholdInMinute', 'timestamp': 1730148814462},
  {'newValue': 120,
   'setting': 'thresholdInMinute',
   'timestamp': 1730148814576},
  {'newValue': 12, 'setting': 'thresholdInMinute', 'timestamp': 1730148816541},
  {'newValue': 2, 'setting': 'messageFrequency', 'timestamp': 1730148058639},
  {'newValue': 0, 'setting': 'messageFrequency', 'timestamp': 1730148059841},
  {'newValue': 0, 'setting': 'thresholdInMinute', 'timestamp': 1730148062582},
  {'newValue': 30, 'setting':

In [ ]:
all_data = []
for key, values in data_dict.items():
    for value in values:
        if value is not None:
            value_id = value.copy()
            value_id['ID'] = key
            all_data.append(value_id)
df_pilot = pd.DataFrame(all_data)
df_pilot = df_pilot[['ID', 'setting', 'timestamp', 'newValue']].rename(columns={'timestamp':'Raw_timestamp'})
df_pilot

,ID,setting,Raw_timestamp,newValue
0,te001,isColorEnabled,1730148797953,1
1,te001,isYellowEnabled,1730148807839,1
2,te001,isYellowEnabled,1730148809787,0
3,te001,isColorEnabled,1730148810194,0
4,te001,thresholdInMinute,1730148813022,3
5,te001,thresholdInMinute,1730148813671,0
6,te001,thresholdInMinute,1730148814462,10
7,te001,thresholdInMinute,1730148814576,120
8,te001,thresholdInMinute,1730148816541,12
9,te001,messageFrequency,1730148058639,2


In [ ]:
df_pilot['timestamp'] = df_pilot['Raw_timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000))  #Time format
df_pilot

,ID,setting,Raw_timestamp,newValue,timestamp
0,te001,isColorEnabled,1730148797953,1,2024-10-28 16:53:17.953
1,te001,isYellowEnabled,1730148807839,1,2024-10-28 16:53:27.839
2,te001,isYellowEnabled,1730148809787,0,2024-10-28 16:53:29.787
3,te001,isColorEnabled,1730148810194,0,2024-10-28 16:53:30.194
4,te001,thresholdInMinute,1730148813022,3,2024-10-28 16:53:33.022
5,te001,thresholdInMinute,1730148813671,0,2024-10-28 16:53:33.671
6,te001,thresholdInMinute,1730148814462,10,2024-10-28 16:53:34.462
7,te001,thresholdInMinute,1730148814576,120,2024-10-28 16:53:34.576
8,te001,thresholdInMinute,1730148816541,12,2024-10-28 16:53:36.541
9,te001,messageFrequency,1730148058639,2,2024-10-28 16:40:58.639


## Import data from Qualtrics

In [ ]:
import requests
import zipfile
import json
import time
import io

def get_qualtrics_data(api_token, survey_id):
    # API configurations
    base_url = f"https://iad1.qualtrics.com/API/v3/surveys/{survey_id}/export-responses"
    headers = {
        "X-API-TOKEN": api_token,
        "Content-Type": "application/json"
    }

    # Start export
    export_payload = '{"format":"csv"}'
    export_response = requests.post(base_url, data=export_payload, headers=headers)
    export_progress_id = export_response.json()["result"]["progressId"]

    # Check export progress
    while True:
        progress_response = requests.get(
            f"{base_url}/{export_progress_id}",
            headers=headers
        )
        progress_status = progress_response.json()["result"]["status"]

        if progress_status == "complete":
            file_id = progress_response.json()["result"]["fileId"]
            break
        time.sleep(2)

    # Download file
    download_response = requests.get(
        f"{base_url}/{file_id}/file",
        headers=headers
    )

    # Extract zip file
    with zipfile.ZipFile(io.BytesIO(download_response.content)) as zip_file:
        return zip_file.read(zip_file.namelist()[0]).decode('utf-8')


In [ ]:
sign_up_survey = "SV_dgN8IwiCIfglbAq"

The API is not enabled for your account. Please contact your administrator For support please contact - itsupport@umd.edu.

Sent an email to itsupport and waiting for their reply.

## Working on CSV file

In [ ]:
import pandas as pd

sign_up_data = pd.read_csv('/content/Sign-up survey.csv')
sign_up_data

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q13,Q13_4_TEXT,Q14,Q15,Q16,Q16_4_TEXT,Q16_8_TEXT,Q18,Q19,Q_DataPolicyViolations
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,How do you describe your gender? - Selected Ch...,How do you describe your gender? - Prefer to s...,What is the highest level of education you hav...,Choose one or more races that you consider you...,Which best describes your current employment s...,Which best describes your current employment s...,Which best describes your current employment s...,By what name would you like me to call you (th...,What is your email?,Q_DataPolicyViolations
1,"{""ImportId"":""startDate"",""timeZone"":""America/Ne...","{""ImportId"":""endDate"",""timeZone"":""America/New_...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID17""}","{""ImportId"":""QID17_4_TEXT""}","{""ImportId"":""QID20""}","{""ImportId"":""QID24""}","{""ImportId"":""QID23""}","{""ImportId"":""QID23_4_TEXT""}","{""ImportId"":""QID23_8_TEXT""}","{""ImportId"":""QID25_TEXT""}","{""ImportId"":""QID26_TEXT""}","{""ImportId"":""Q_DataPolicyViolations""}"
2,2024-10-28 17:27:50,2024-10-28 17:27:54,Survey Preview,NaN,100,4,True,2024-10-28 17:27:55,R_6KHDkdeU38MoZj3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-10-28 17:31:43,2024-10-28 17:31:47,Survey Preview,NaN,100,4,True,2024-10-28 17:31:48,R_6Kw4fLiY5gAR8ug,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-10-28 17:35:43,2024-10-28 17:35:48,Survey Preview,NaN,100,4,True,2024-10-28 17:35:48,R_6vWQH88MmG10dcB,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-10-28 17:35:50,2024-10-28 17:35:55,Survey Preview,NaN,100,4,True,2024-10-28 17:35:55,R_6pJlZWMfjLdP59U,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-10-28 17:46:11,2024-10-28 17:46:26,Survey Preview,NaN,100,14,True,2024-10-28 17:46:26,R_6X0yYeEUOD73mCa,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-10-28 19:21:55,2024-10-28 19:22:00,Survey Preview,NaN,100,5,True,2024-10-28 19:22:01,R_61X6C9mZZ3Hzth7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-10-28 19:24:37,2024-10-28 19:24:41,Survey Preview,NaN,100,4,True,2024-10-28 19:24:41,R_6BqBMhKTWDVibYd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-10-28 19:24:50,2024-10-28 19:24:56,Survey Preview,NaN,100,5,True,2024-10-28 19:24:56,R_6tISpwXZKTTl1zH,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Paricipant management

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Progress check

Which day is each participant on, the state of each survey and the tutorial: unavailable, available, or completed.

Generate a list of emails for certain filter (e.g., Survey2 available)

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

We need a list of participants who we are going to send the reminder emails to.